# 話者認識システム - サンプルノートブック

このノートブックでは、話者認識システムの主要な機能を試すことができます。

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display

# プロジェクトのルートディレクトリをパスに追加
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# プロジェクトの自作モジュールをインポート
from src.features.extract_features import extract_mfcc, extract_features_from_dir
from src.utils.audio_utils import load_audio, plot_waveform, plot_spectrogram

## 1. 音声データの確認

まず、話者のサンプル音声を読み込んで確認します。

In [ ]:
# データディレクトリ
data_dir = project_root / "data"

# 話者ディレクトリを取得
speaker_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d)) and not d.startswith('.')]
print(f"話者一覧: {speaker_dirs}")

In [ ]:
# 最初の話者の最初の音声ファイルを読み込む
speaker_dir = os.path.join(data_dir, speaker_dirs[0])
audio_files = [f for f in os.listdir(speaker_dir) if f.endswith(('.wav', '.mp3'))]

if audio_files:
    audio_file = os.path.join(speaker_dir, audio_files[0])
    print(f"読み込む音声ファイル: {audio_file}")
    
    # 音声の読み込み
    y, sr = load_audio(audio_file)
    
    if y is not None:
        print(f"音声データ長さ: {len(y)} サンプル")
        print(f"サンプリングレート: {sr} Hz")
        print(f"音声長: {len(y)/sr:.2f} 秒")
        
        # 波形の表示
        plot_waveform(y, sr, title=f"{speaker_dirs[0]}の波形")
        
        # スペクトログラムの表示
        plot_spectrogram(y, sr, title=f"{speaker_dirs[0]}のスペクトログラム")
    else:
        print("音声の読み込みに失敗しました")
else:
    print(f"{speaker_dir} には音声ファイルがありません")

## 2. 特徴量抽出

音声から特徴量（MFCC）を抽出します。

In [ ]:
# 特徴量の抽出
if 'audio_file' in locals() and y is not None:
    features = extract_mfcc(audio_file)
    print(f"抽出された特徴量の形状: {features.shape}")
    
    # 特徴量の可視化
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(features)), features)
    plt.title("MFCC特徴量")
    plt.xlabel("特徴量インデックス")
    plt.ylabel("値")
    plt.tight_layout()
    plt.show()

## 3. 全データからの特徴量抽出

すべての話者のデータから特徴量を抽出します。

In [ ]:
# すべてのデータから特徴量を抽出
features_file = data_dir / "features.pkl"

# 特徴量ファイルが存在しない場合は抽出する
if not os.path.exists(features_file):
    print("特徴量を抽出します...")
    features_df = extract_features_from_dir(data_dir, features_file)
else:
    print("既存の特徴量ファイルを読み込みます...")
    features_df = pd.read_pickle(features_file)

print(f"特徴量データフレームの形状: {features_df.shape}")
print("\n最初の数行:")
display(features_df.head())

# 話者ごとのサンプル数
speaker_counts = features_df['speaker'].value_counts()
print("\n話者ごとのサンプル数:")
display(speaker_counts)

## 4. 特徴量の可視化

抽出された特徴量を可視化します。

In [ ]:
# 特徴量の可視化（次元削減）
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# 特徴量の列だけを取得
feature_cols = [col for col in features_df.columns if col.startswith('feature_')]
X = features_df[feature_cols].values
y = features_df['speaker'].values

# PCAで2次元に削減
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 8))
for speaker in np.unique(y):
    plt.scatter(X_pca[y==speaker, 0], X_pca[y==speaker, 1], label=speaker, alpha=0.7)
plt.legend()
plt.title("PCAによる特徴量の可視化")
plt.xlabel("第1主成分")
plt.ylabel("第2主成分")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# t-SNEで2次元に削減（データが多い場合は時間がかかるので注意）
if len(X) < 500:  # サンプル数が多すぎる場合はスキップ
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X)
    
    plt.figure(figsize=(10, 8))
    for speaker in np.unique(y):
        plt.scatter(X_tsne[y==speaker, 0], X_tsne[y==speaker, 1], label=speaker, alpha=0.7)
    plt.legend()
    plt.title("t-SNEによる特徴量の可視化")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

## 5. モデルの訓練

特徴量を使って話者認識モデルを訓練します。

In [ ]:
# モデルのトレーニング
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# データの分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 特徴量のスケーリング
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# モデルのトレーニング
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
print("モデルのトレーニングが完了しました")

# モデルの評価
y_pred = model.predict(X_test)
print("\n分類レポート:")
print(classification_report(y_test, y_pred))

# 混同行列の表示
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=np.unique(y_test), 
            yticklabels=np.unique(y_test))
plt.xlabel('予測')
plt.ylabel('真値')
plt.title('混同行列')
plt.tight_layout()
plt.show()

## 6. 特徴量の重要度

ランダムフォレストの特徴量重要度を確認します。

In [ ]:
# 特徴量の重要度
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(min(30, len(importances))), importances[indices[:30]], align='center')
plt.xticks(range(min(30, len(importances))), indices[:30])
plt.xlabel('特徴量インデックス')
plt.ylabel('重要度')
plt.title('上位30の特徴量重要度')
plt.tight_layout()
plt.show()